## Horizontal Counter-Flow Assessment
### Prepared By : Md Rasel
### Matriculation No. 2130710
I have used "Lane formation in bi-directional flow" from jupedsim's website as my starting point. In the methodology of the step-1,it was mentioned that the space should be filled with the highest possible density. However, I am not sure how can that be implemented.
In the step-2, when the agents were added to the simulation, for 10 agents, the evacuation time was over 2 times than before (step-1). Moreover, my simulation was going on for an infinte period of time while the agents on room2 were 50 and 100. Due to this reason, I have stoped the simulation after 3000 iterations.

In [ ]:
import pathlib
import jupedsim as jps
from shapely import Polygon, GeometryCollection
import pedpy
import matplotlib.pyplot as plt

### Geometry

In [ ]:
room1 = Polygon([(0, 0), (10, 0), (10, 10), (0, 10)])
room2 = Polygon([(20, 0), (30, 0), (30, 10), (20, 10)])
corridor = Polygon([(10, 4), (20, 4), (20, 6), (10, 6)])
area = GeometryCollection(corridor.union(room1.union(room2)))

exit_polygon_left = Polygon([(9.9, 4), (9.8, 4), (9.8, 6), (9.9, 6)])
exit_polygon_right = Polygon([(20.1, 4), (20.2, 4), (20.2, 6), (20.1, 6)])

distribution_polygon_left = Polygon([(0, 0), (5, 0), (5, 10), (0, 10)])
distribution_polygon_right = Polygon([(25, 0), (30, 0), (30, 10), (25, 10)])

measurement_area = pedpy.MeasurementArea([[10, 4], [20, 4], [20, 6], [10, 6]])
measurement_line_left = pedpy.MeasurementLine([[10, 4], [10, 6]])
measurement_line_right = pedpy.MeasurementLine([[20, 4], [20, 6]])
walkable_area = pedpy.WalkableArea(area.geoms[0])

fig, ax = plt.subplots(nrows=1, ncols=1)
ax.set_aspect("equal")
pedpy.plot_measurement_setup(
    walkable_area=walkable_area,
    measurement_areas=[measurement_area],
    measurement_lines=[measurement_line_left, measurement_line_right],
    ml_color="red",
    ml_width=2,
    axes=ax,
)
for id, polygon in enumerate(
    [distribution_polygon_left, distribution_polygon_right]
):
    x, y = polygon.exterior.xy
    plt.fill(x, y, alpha=0.1, color="gray")
    centroid = polygon.centroid
    plt.text(
        centroid.x,
        centroid.y,
        f"Room {id+1}",
        ha="center",
        va="center",
        fontsize=10,
    )

### Simulation definition, scenarios

In [ ]:
simulations = {}
COLUMNS = 10
number_agents = [
    (10 * COLUMNS, 0 * COLUMNS), # 100 agents in room1
    (10 * COLUMNS, 1 * COLUMNS), # 100 agents in room1, 10 agents in room2
    (10 * COLUMNS, 5 * COLUMNS), # 100 agents in room1, 50 agents in room2
    (10 * COLUMNS, 10 * COLUMNS), # 100 agents in room1, 100 agents in room2
]
for number in number_agents:
    trajectory_file = f"trajectories_number_agents_{number}.sqlite"
    simulation = jps.Simulation(
        dt=0.05,
        model=jps.CollisionFreeSpeedModel(
            strength_neighbor_repulsion=2.6,
            range_neighbor_repulsion=0.1,
            range_geometry_repulsion=0.05,
        ),
        geometry=walkable_area.polygon,
        trajectory_writer=jps.SqliteTrajectoryWriter(
            output_file=pathlib.Path(trajectory_file),
        ),
    )
    simulations[number] = simulation

rhs_agents_variable = {}
lhs_agents_fixed = {}

for number, simulation in simulations.items():
    exits = [
        simulation.add_exit_stage(exit_polygon_left),
        simulation.add_exit_stage(exit_polygon_right),
    ]
    journeys = [
        simulation.add_journey(jps.JourneyDescription([exit]))
        for exit in exits
    ]

    # right side group
    positions = jps.distribute_by_number(
        polygon=distribution_polygon_right,
        number_of_agents=number[1],
        distance_to_agents=0.4,
        distance_to_polygon=0.5,
        seed=45131502,
    )
    group1 = set(
        [
            simulation.add_agent(
                jps.CollisionFreeSpeedModelAgentParameters(
                    position=position,
                    journey_id=journeys[0],
                    stage_id=exits[0],
                )
            )
            for position in positions
        ]
    )

    # left side group
    positions = jps.distribute_by_number(
        polygon=distribution_polygon_left,
        number_of_agents=number[0],
        distance_to_agents=0.4,
        distance_to_polygon=0.5,
        seed=45131502,
    )

    group2 = set(
        [
            simulation.add_agent(
                jps.CollisionFreeSpeedModelAgentParameters(
                    position=position,
                    journey_id=journeys[1],
                    stage_id=exits[1],
                )
            )
            for position in positions
        ]
    )

    rhs_agents_variable[number] = group1
    lhs_agents_fixed[number] = group2
    
trajectory_files = {}
dt = 0.05
for number, simulation in simulations.items():
    while simulation.agent_count() > 0 and simulation.iteration_count() < 3000:
        simulation.iterate()
    
    print(
        f"Simulation time: {simulation.iteration_count()*dt} seconds {number} agents."
    )

    trajectory_file = f"trajectories_number_agents_{number}.sqlite"
    trajectory_files[number] = trajectory_file

### Visualisation 

In [ ]:
from jupedsim.internal.notebook_utils import animate, read_sqlite_file

agent_trajectories = {}
for number in number_agents:
    trajectory_file = trajectory_files[number]
    agent_trajectories[number], walkable_area = read_sqlite_file(trajectory_file)
    animate(agent_trajectories[number],walkable_area,every_nth_frame=5).show()

### Data Visualisation

In [ ]:
evac_times = {}
for i, number in enumerate(number_agents):
    trajectories = agent_trajectories[number].data
    fps = agent_trajectories[number].frame_rate
    evac_time = trajectories["frame"].max() / fps
    agents_room2 = number[1]
    evac_times[agents_room2] = evac_time

plt.plot(list(evac_times.keys()), list(evac_times.values()), "o--", ms=10)
plt.xlabel("Number of agents in Room2")
plt.ylabel("Evacuation time [s]")
plt.grid()
plt.show()